# Task 1.3. Запрос на поиск облигаций
Необходимо последовательно вводить SQL-запросы

Создадим отдельную таблицу с данными о торгах с сортировкой по времени и ID, необходимыми для выполнения задания 

In [ ]:
ALTER TABLE [risks].[dbo].[base_prices] ALTER COLUMN [TIME] INT;
SELECT * INTO [risks].[dbo].[task.1.3] FROM [risks].[dbo].[base_prices]	
ORDER BY [TIME];
SELECT ID, [TIME], ISIN, ASK, LAG(ASK,1,0) OVER (ORDER BY ID) AS ASK_prev, LAG(ID,1,0) OVER (ORDER BY ID) AS ID_prev
INTO [risks].[dbo].[task.1.3_1] FROM [risks].[dbo].[task.1.3]
ORDER BY [ID];
GO

Удалим данные без нужной нам информации

In [ ]:
DELETE FROM [risks].[dbo].[task.1.3_1] WHERE ID != ID_prev AND ASK != ' ';
GO

Создадим две вспомогательные таблицы, где для каждой ценной бумаги будет информация, сколько дней не было торгов, и по скольким есть

In [ ]:
SELECT ID, COUNT([TIME]) AS n_days 
INTO [risks].[dbo].[task.1.3_2] FROM [risks].[dbo].[task.1.3_1] 
WHERE ASK=ASK_prev OR ASK=' ' GROUP BY ID;
SELECT ID, COUNT([TIME]) AS N
INTO [risks].[dbo].[task.1.3_3] FROM [risks].[dbo].[task.1.3_1] 
GROUP BY ID;
GO

Запустим скрипт, создающий таблицу с ценными бумагами, по которым было не более 10% дней без торгов

In [ ]:
SELECT [risks].[dbo].[task.1.3_3].[ID],n_days, N, n_days*100/N AS share
INTO [risks].[dbo].[task.1.3_4]
FROM [risks].[dbo].[task.1.3_2] RIGHT JOIN [risks].[dbo].[task.1.3_3]
ON [risks].[dbo].[task.1.3_3].[ID]= [risks].[dbo].[task.1.3_2].[ID]
WHERE [risks].[dbo].[task.1.3_2].[n_days] IS NULL OR n_days*100/N <= 10; 
SELECT [risks].[dbo].[task.1.3_4].[ID], Exchange, CFIName, EmitentName
INTO [risks].[dbo].[task.1.3_fin]
FROM [risks].[dbo].[task.1.3_4] INNER JOIN [risks].[dbo].[instrs]
ON [risks].[dbo].[task.1.3_4].[ID]= [risks].[dbo].[instrs].[ID]
WHERE Exchange = 'Московская Биржа / МБ - Основной' 
AND (CFIName = 'Облигации / Сектор / Корпоративные' OR CFIName = 'Облигации / Сектор / Региональные');  
GO

Получили 11 облигаций

In [ ]:
#первые 3 облигации

SELECT TOP(3) EmitentName
FROM [risks].[dbo].[task.1.3_fin]
GROUP BY EmitentName;
GO

In [ ]:
Результат:
Аптечная сеть 36.6
ПГК, АО
РЖД-Развитие вокзалов